# LSTM con embeddings + double translated

Modelo básico con los embeddings de fasttext

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


In [2]:
train_synth_en = pd.read_table("../../../data/es/train_es.synth.en.tsv", index_col="id", quoting=csv.QUOTE_NONE)
train_synth_fr = pd.read_table("../../../data/es/train_es.synth.fr.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train_synth_en, y_train_synth_en = train_synth_en["text"], train_synth_en["HS"]
text_train_synth_fr, y_train_synth_fr = train_synth_fr["text"], train_synth_fr["HS"]

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

X_train_synth_en = tokenizer.texts_to_sequences(text_train_synth_en)
X_train_synth_fr = tokenizer.texts_to_sequences(text_train_synth_fr)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)
X_train_synth_en = pad_sequences(X_train_synth_en, max_length)
X_train_synth_fr = pad_sequences(X_train_synth_fr, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [4]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("~/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [5]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [6]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
embedding_matrix.shape

(200000, 300)

# Sin Synths


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 8s 2ms/step - loss: 0.6210 - acc: 0.6702 - val_loss: 0.5703 - val_acc: 0.7080
Epoch 2/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.5394 - acc: 0.7378 - val_loss: 0.5382 - val_acc: 0.7220
Epoch 3/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.4974 - acc: 0.7718 - val_loss: 0.5282 - val_acc: 0.7440
Epoch 4/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.4783 - acc: 0.7773 - val_loss: 0.5124 - val_acc: 0.7600
Epoch 5/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.4574 - acc: 0.7916 - val_loss: 0.5151 - val_acc: 0.7720
Epoch 6/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.4452 - acc: 0.8018 - val_loss: 0.5064 - val_acc: 0.7700
Epoch 7/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.4302 - acc: 0.8127 - val_loss: 0.5175 - val_acc: 0.7740
Epoch 8/20
4500

In [9]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 627us/step
Loss        : 0.5251
Accuracy    : 0.7780
Precision   : 0.7467
Recall      : 0.7568
F1          : 0.7517


## Usando los synths generados de entrenamiento (sólo inglés)

In [10]:

X_tr = np.vstack((X_train, X_train_synth_en))
y_tr = np.vstack((y_train.values.reshape(-1, 1), 
                  y_train_synth_en.values.reshape(-1, 1),
))

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_tr, y_tr, 
          validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 5991 samples, validate on 500 samples
Epoch 1/20
5991/5991 [==============================] - 10s 2ms/step - loss: 0.6176 - acc: 0.6753 - val_loss: 0.5596 - val_acc: 0.7220
Epoch 2/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.5335 - acc: 0.7478 - val_loss: 0.5455 - val_acc: 0.7360
Epoch 3/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.5026 - acc: 0.7675 - val_loss: 0.5200 - val_acc: 0.7560
Epoch 4/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.4820 - acc: 0.7813 - val_loss: 0.5184 - val_acc: 0.7540
Epoch 5/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.4668 - acc: 0.7852 - val_loss: 0.5072 - val_acc: 0.7620
Epoch 6/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.4555 - acc: 0.7895 - val_loss: 0.5084 - val_acc: 0.7720
Epoch 7/20
5991/5991 [==============================] - 9s 2ms/step - loss: 0.4470 - acc: 0.8004 - val_loss: 0.5113 - val_acc: 0.7680
Epoch 8/20
599

In [12]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 600us/step
Loss        : 0.5081
Accuracy    : 0.7900
Precision   : 0.7600
Recall      : 0.7703
F1          : 0.7651


## Usando Inglés + Francés

In [13]:

X_tr = np.vstack((X_train, X_train_synth_en, X_train_synth_fr))
y_tr = np.vstack((
    y_train.values.reshape(-1, 1), 
    y_train_synth_en.values.reshape(-1, 1),
    y_train_synth_fr.values.reshape(-1, 1)
))

In [14]:

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_tr, y_tr, 
          validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 7168 samples, validate on 500 samples
Epoch 1/20
7168/7168 [==============================] - 11s 2ms/step - loss: 0.6146 - acc: 0.6738 - val_loss: 0.5677 - val_acc: 0.7140
Epoch 2/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.5353 - acc: 0.7457 - val_loss: 0.5413 - val_acc: 0.7220
Epoch 3/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.5004 - acc: 0.7617 - val_loss: 0.5312 - val_acc: 0.7360
Epoch 4/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.4829 - acc: 0.7790 - val_loss: 0.5306 - val_acc: 0.7460
Epoch 5/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.4710 - acc: 0.7832 - val_loss: 0.5176 - val_acc: 0.7500
Epoch 6/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.4556 - acc: 0.7948 - val_loss: 0.5120 - val_acc: 0.7580
Epoch 7/20
7168/7168 [==============================] - 11s 1ms/step - loss: 0.4483 - acc: 0.7981 - val_loss: 0.5143 - val_acc: 0.7640
Epoch 8/

In [15]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 581us/step
Loss        : 0.5467
Accuracy    : 0.7580
Precision   : 0.6980
Recall      : 0.8018
F1          : 0.7463
